## Setup Azure OpenAI

We'll start as usual by defining our Azure OpenAI service API key and endpoint details, specifying the model deployment we want to use and then we'll initiate a connection to the Azure OpenAI service.

**NOTE**: As with previous labs, we'll use the values from the `.env` file in the root of this repository.

In [1]:
import os
from dotenv import load_dotenv

 # Load environment variables
if load_dotenv():
    print("Found OpenAPI Base Endpoint: " + os.getenv("OPENAI_API_BASE"))
else: 
    print("No file .env found")
openai_api_type = os.getenv("OPENAI_API_TYPE")
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_base = os.getenv("OPENAI_API_BASE")
openai_api_version = os.getenv("OPENAI_API_VERSION")
deployment_name = os.getenv("OPENAI_DEPLOYMENT_NAME")
embedding_name = os.getenv("OPENAI_EMBEDDING_DEPLOYMENTE")
acs_service_name = os.getenv("AZURE_SEARCH_SERVICE_NAME")
acs_endpoint_name = os.getenv("AZURE_SEARCH_ENDPOINT")
acs_index_name = "movies-index-hzm"
acs_api_key = os.getenv("AZURE_SEARCH_KEY")

Found OpenAPI Base Endpoint: https://trefoil.openai.azure.com/


### Inititate  the Embedding model, the completion and the instrcut model. 

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI

# Create an Embeddings Instance of Azure OpenAI

embeddings = OpenAIEmbeddings(
    openai_api_base = openai_api_base,
    openai_api_version = openai_api_version,
    deployment_name ="text-embedding-ada-002",
    openai_api_key = openai_api_key,
    openai_api_type = openai_api_type,
    embedding_ctx_length=8191,
    chunk_size=1000,
    max_retries=6)

# Create a Completion Instance of Azure OpenAI
llm = AzureChatOpenAI(
    openai_api_base= openai_api_base,
    openai_api_version= openai_api_version,
    deployment_name="gpt-35-turbo-16k",
    temperature=0,
    openai_api_key= openai_api_key,
    openai_api_type = openai_api_type,
    max_retries=6,
    max_tokens=4000
)

llmi = AzureOpenAI(
    openai_api_base= openai_api_base,
    openai_api_version= openai_api_version,
    deployment_name="gpt-35-turbo-instruct",
    temperature=0,
    openai_api_key= openai_api_key,
    openai_api_type = openai_api_type,
    max_retries=6,
    max_tokens=4000
)
print('Completed creation of embedding and completion instances.')

Completed creation of embedding and completion instances.


#### Load the data from the movies.csv file using the Langchain CSV document loader.

In [3]:
from langchain.document_loaders.csv_loader import CSVLoader

# Movie Fields in CSV
# id,original_language,original_title,popularity,release_date,vote_average,vote_count,genre,overview,revenue,runtime,tagline
loader = CSVLoader(file_path='../data/movies.csv', source_column='original_title', encoding='utf-8', 
                   csv_args={'delimiter':',', 
                             'fieldnames': ['id', 'original_language', 'original_title', 'popularity', 
                                            'release_date', 'vote_average', 'vote_count', 'genre', 
                                            'overview', 'revenue', 'runtime', 'tagline'
                                            ]
                            }
                    )
data = loader.load()
data = data[1:101] # reduce dataset if you want
print('Loaded %s movies' % len(data))
data[0]

Loaded 100 movies


Document(page_content="id: 381284.0\noriginal_language: en\noriginal_title: Hidden Figures\npopularity: 49.802\nrelease_date: 2016-12-10\nvote_average: 8.1\nvote_count: 7310.0\ngenre: ['Drama', 'History']\noverview: The untold story of Katherine G. Johnson, Dorothy Vaughan and Mary Jackson – brilliant African-American women working at NASA and serving as the brains behind one of the greatest operations in history – the launch of astronaut John Glenn into orbit. The visionary trio crossed all gender and race lines to inspire generations to dream big.\nrevenue: 230698791.0\nruntime: 127.0\ntagline: Meet the women you don't know, behind the mission you do.", metadata={'source': 'Hidden Figures', 'row': 1})

### Create embedings for every entry/row in our `data` object and put everything in an object called Items 

In [6]:
import uuid # The uuid library in Python is used to generate unique IDs, known as Universally Unique Identifiers (UUIDs), which can be used for objects, sessions, or transactions where uniqueness is required.

# Let's take a quick look at the data structure of the CSVLoader
print(data[0])
print(data[0].metadata['source'])
print("----------")

# Generate Document Embeddings for page_content field in the movies CSVLoader dataset using Azure OpenAI
items = []
for movie in data:
    content = movie.page_content
    items.append(dict([("id", str(uuid.uuid4())), ("original_title", movie.metadata['source']), ("content", content), ("content_vector", embeddings.embed_query(content))]))

# Print out a sample item to validate the updated data structure.
# It should have the id, content, and content_vector values.
print(items[0])

page_content="id: 381284.0\noriginal_language: en\noriginal_title: Hidden Figures\npopularity: 49.802\nrelease_date: 2016-12-10\nvote_average: 8.1\nvote_count: 7310.0\ngenre: ['Drama', 'History']\noverview: The untold story of Katherine G. Johnson, Dorothy Vaughan and Mary Jackson – brilliant African-American women working at NASA and serving as the brains behind one of the greatest operations in history – the launch of astronaut John Glenn into orbit. The visionary trio crossed all gender and race lines to inspire generations to dream big.\nrevenue: 230698791.0\nruntime: 127.0\ntagline: Meet the women you don't know, behind the mission you do." metadata={'source': 'Hidden Figures', 'row': 1}
Hidden Figures
----------
{'id': '9f50f113-ecbb-4e6c-a84e-0f464b64cc6c', 'original_title': 'Hidden Figures', 'content': "id: 381284.0\noriginal_language: en\noriginal_title: Hidden Figures\npopularity: 49.802\nrelease_date: 2016-12-10\nvote_average: 8.1\nvote_count: 7310.0\ngenre: ['Drama', 'Histo

In [7]:
content = movie.page_content
print(content)

id: 419743.0
original_language: en
original_title: Disobedience
popularity: 41.103
release_date: 2017-09-10
vote_average: 7.0
vote_count: 866.0
genre: ['Drama', 'Romance']
overview: A woman learns about the death of her Orthodox Jewish father, a rabbi. She returns home and has romantic feelings rekindled for her best childhood friend, who is now married to her cousin.
revenue: 0.0
runtime: 114.0
tagline: Love is an act of defiance


In [8]:
print(embeddings.embed_query(content))

[0.00025464172334035886, -0.030267897964098858, -0.03032137450628164, -0.019251666944510613, 0.006183261045803336, 0.024305229983105935, 0.0075001283476349495, -0.0004591488973460547, -0.02304852379122995, -0.03197915848981879, 0.02010729534472543, 0.031203743040232997, -0.003340632185839432, -0.01564197985807616, 0.005641807796573653, 0.012647275800711429, 0.03759422522133333, -0.009980117892522676, 0.0038302794527529356, 0.003193570763514206, 0.005130435335152429, 0.023543184600388414, -0.01826234532619369, -0.006801587057251412, 0.017206177098825718, 0.0007073149311045521, 0.0210966188256875, -0.01727302370787677, -0.00035762644221931964, -0.034866906668850235, -0.016845207645124212, 0.0034693106975433614, -0.002177510641275249, -0.019305143486693394, -0.043209295678138165, -0.021858662345759873, -0.014679395579528055, -0.026110073526323163, 0.032620880721302474, -0.0011856820190052051, 0.011009546917476685, -0.003693245114917625, -0.015789041280401386, -0.006972044699612249, -0.012

### Load Movies into Azure Cognitive Search

Next, we'll create the Azure Cognitive Search index, embed the loaded movies from the CSV file, and upload the data into the newly created index. Depending on the number of movies loaded and rate limiting, this might take a while to do the embeddings so be patient.

In [9]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    SearchIndex,
    SemanticConfiguration,
    PrioritizedFields,
    SemanticField,
    SemanticSettings,
    VectorSearch,
    HnswVectorSearchAlgorithmConfiguration,
)

### Let's Create the Azure Cognitive Search Index Client

In [10]:
index_client = SearchIndexClient(
    acs_endpoint_name,
    AzureKeyCredential(acs_api_key)
)

###  Define search carachteristics for the movie's fields in the CSV

In [11]:
# Fields in the csv file
# id,original_language,original_title,popularity,release_date,vote_average,vote_count,genre,overview,revenue,runtime,tagline
# Definition of the structure of the index. 

fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="original_title", type=SearchFieldDataType.String),
    SearchableField(name="overview", type=SearchFieldDataType.String),
    SearchableField(name="tagline", type=SearchFieldDataType.String),
    SearchableField(name="popularity", type=SearchFieldDataType.Double, sortable=True),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchField(name="content_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
]

### Configure Vector Search Configuration

In [12]:
vector_search = VectorSearch(
    algorithm_configurations=[
        HnswVectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 500,
                "metric": "cosine"
            }
        )
    ]
)

### Configure Semantic Configuration (work in progress)

In [13]:
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="original_title"),
        prioritized_keywords_fields=[SemanticField(field_name="original_title"), SemanticField(field_name="tagline")],
        prioritized_content_fields=[SemanticField(field_name="content")]
    )
)


### Create the semantic settings with the configuration

In [14]:
# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

### Create the search index with the desired vector search and semantic configuration

In [15]:
# Create the search index with the desired vector search and semantic configurations
index = SearchIndex(
    name=acs_index_name,
    fields=fields,
    vector_search=vector_search,
    semantic_settings=semantic_settings
)
result = index_client.create_or_update_index(index)
print(f'The {result.name} index was created.')

The movies-index-hzm index was created.


### Upload movies to Azure Cognitive Search index.

In [16]:
from azure.search.documents.models import Vector
from azure.search.documents import SearchClient

# Insert Text and Embeddings into the Azure Cognitive Search index created.
search_client = SearchClient(
    acs_endpoint_name,
    acs_index_name,
    AzureKeyCredential(acs_api_key)
)
result = search_client.upload_documents(items)
print("Successfully added documents to Azure Cognitive Search index.")
print(f"Uploaded {len(data)} documents")

Successfully added documents to Azure Cognitive Search index.
Uploaded 100 documents


### First, let's do a plain vanilla text search, no vectors or embeddings.

In [17]:
query = "old movie?"

search_client = SearchClient(
    acs_endpoint_name,
    acs_index_name,
    AzureKeyCredential(acs_api_key)
)

# Execute the search
results = list(search_client.search(
    search_text=query,
    include_total_count=True,
    top=3
))

# Print count of total results.
print(f"Returned {len(results)} results using only text-based search.")
print("----------")
# Iterate over Results
# Index Fields - id, content, content_vector
for result in results:
    print("Movie: {}".format(result["content"]))
    print("----------")

Returned 3 results using only text-based search.
----------
Movie: id: 27098.0
original_language: sv
original_title: Lust och fägring stor
popularity: 17.826
release_date: 1995-03-08
vote_average: 6.5
vote_count: 73.0
genre: ['Drama', 'Romance', 'War']
overview: Stig is a 15-year-old pupil of 37-year-old teacher Viola. He is attracted by her beauty and maturity while she is drawn to him by his youth and innocence, a godsent relief from her drunk and miserable husband.
revenue: 0.0
runtime: 130.0
tagline: He was a student. She was his teacher. Their love was forbidden.
----------
Movie: id: 6552.0
original_language: en
original_title: Idle Hands
popularity: 12.326
release_date: 1999-04-30
vote_average: 6.1
vote_count: 452.0
genre: ['Comedy', 'Horror', 'Fantasy']
overview: Anton is a cheerful but exceedingly non-ambitious 17-year-old stoner who lives to stay buzzed, watch TV, and moon over Molly, the beautiful girl who lives next door. However, it turns out that the old cliché about idle

### Now let's do a vector search that uses the embeddings we created and inserted into `content_vector` field in the index.

In [18]:
query = "What are the best 80s movies I should look at?"

search_client = SearchClient(
    acs_endpoint_name,
    acs_index_name,
    AzureKeyCredential(acs_api_key)
)

# You can see here that we are getting the embedding representation of the query.
vector = Vector(
    value=embeddings.embed_query(query),
    k=5,
    fields="content_vector"
)

# Execute the search
results = list(search_client.search(
    search_text="",
    include_total_count=True,
    vectors=[vector],
    select=["id", "content", "original_title"],
))

# Print count of total results.
print(f"Returned {len(results)} results using only vector-based search.")
print("----------")
# Iterate over results and print out the content.
for result in results:
    print(result['original_title'])
    print("----------")

Returned 5 results using only vector-based search.
----------
America: The Motion Picture
----------
Enemy of the State
----------
Evolution
----------
Work It
----------
Spy Kids 3-D: Game Over
----------


Did that return what you expected? Probably not, let's dig deeper to see why. Let's do the same search again, but this time let's return the `search score` so we can see the value returned by the cosine similarity vector store calculation.

### Try again, but this time let's add the relevance score to maybe see why

In [19]:
query = "What are the best 80s movies I should look at?"


search_client = SearchClient(
    acs_endpoint_name,
    acs_index_name,
    AzureKeyCredential(acs_api_key)
)

# You can see here that we are getting the embedding representation of the query.
vector = Vector(
    value=embeddings.embed_query(query),
    k=5,
    fields="content_vector"
)

# Execute the search
results = list(search_client.search(
    search_text="",
    include_total_count=True,
    vectors=[vector],
    select=["id", "content", "original_title"],
))

# Print count of total results.
print(f"Returned {len(results)} results using vector search.")
print("----------")
# Iterate over results and print out the id and search score.
for result in results:  
    print(f"Id: {result['id']}")
    print(f"Id: {result['original_title']}")
    print(f"Score: {result['@search.score']}")
    print("----------")

Returned 5 results using vector search.
----------
Id: 6f6b152b-6fd6-4f6c-8913-ad09807f6206
Id: America: The Motion Picture
Score: 0.8032411
----------
Id: 5b57538f-c14d-4081-9550-d6da1af6d0e7
Id: Enemy of the State
Score: 0.7995731
----------
Id: 022c202a-f9c5-45f8-be1d-3ecaf4b6a228
Id: Evolution
Score: 0.79799867
----------
Id: efdf94dd-0b38-4ca6-99d1-a8f95ba50ddd
Id: Work It
Score: 0.7959727
----------
Id: 73f33485-11aa-4e79-85b6-92c628620eb1
Id: Spy Kids 3-D: Game Over
Score: 0.79586643
----------


If you look at the `search score` you will see the relevant ranking of the closest vector match to the query inputted. The lower the score the farther apart the two vectors are. Let's change the search term and see if we can get a higher Search Score which means a higher match and closer vector proximity.

### Try again, but this time let's add the relevance score to maybe see why

In [20]:

query =  "Who are the actors in the movie Hidden Figures?"

search_client = SearchClient(
    acs_endpoint_name,
    acs_index_name,
    AzureKeyCredential(acs_api_key)
)

# You can see here that we are getting the embedding representation of the query.
vector = Vector(
    value=embeddings.embed_query(query),
    k=5,
    fields="content_vector"
)

# Execute the search
results = list(search_client.search(
    search_text="",
    include_total_count=True,
    vectors=[vector],
    select=["id", "content", "original_title"],
))

# Print count of total results.
print(f"Returned {len(results)} results using vector search.")
print("----------")
# Iterate over results and print out the id and search score.
for result in results:  
    print(f"Id: {result['id']}")
    print(f"Id: {result['original_title']}")
    print(f"Score: {result['@search.score']}")
    print("----------")
    

Returned 5 results using vector search.
----------
Id: 9f50f113-ecbb-4e6c-a84e-0f464b64cc6c
Id: Hidden Figures
Score: 0.8826789
----------
Id: 591e75dc-b477-48dc-95a9-60be485066fa
Id: I Don't Know How She Does It
Score: 0.80338657
----------
Id: c369ee73-1251-4e4e-bb96-21cde2305289
Id: Step Sisters
Score: 0.8006252
----------
Id: 233354af-807b-4685-812c-902a35ea4a2b
Id: Lovelace
Score: 0.79975164
----------
Id: 0d34d588-8b70-4588-a1ea-d39caf1e1127
Id: Women
Score: 0.79863286
----------


**NOTE:** As you have seen from the results, different inputs can return different results, it all depends on what data is in the Vector Store. The higher the score the higher the likelihood of a match.

### Hybrid Searching using Azure Cognitive Search
What is Hybrid Search? The search is implemented at the field level, which means you can build queries that include vector fields and searchable text fields. The queries execute in parallel and the results are merged into a single response. Optionally, add semantic search, currently in preview, for even more accuracy with L2 reranking using the same language models that power Bing.

**NOTE:** Hybrid Search is a key value proposition of Azure Cognitive Search in comparison to vector only data stores. Click Hybrid Search for more details.

In [21]:
# Hybrid Search
# Let's try our original query again using Hybrid Search (ie. Combination of Text & Vector Search)
query = "What are the best 80s movies I should look at?"

search_client = SearchClient(
    acs_endpoint_name,
    acs_index_name,
    AzureKeyCredential(acs_api_key)
)

# You can see here that we are getting the embedding representation of the query.
vector = Vector(
    value=embeddings.embed_query(query),
    k=5,
    fields="content_vector"
)

# Notice we also fill in the search_text parameter with the query.
results = list(search_client.search(
    search_text=query,
    include_total_count=True,
    top=10,
    vectors=[vector],
    select=["id", "content", "original_title"],
))

# Print count of total results.
print(f"Returned {len(results)} results using vector search.")
print("----------")
# Iterate over results and print out the id and search score.
for result in results:  
    print(f"Id: {result['id']}")
    print(result['original_title'])
    print(f"Hybrid Search Score: {result['@search.score']}")
    print("----------")

Returned 10 results using vector search.
----------
Id: 6f6b152b-6fd6-4f6c-8913-ad09807f6206
America: The Motion Picture
Hybrid Search Score: 0.02857143059372902
----------
Id: 5b57538f-c14d-4081-9550-d6da1af6d0e7
Enemy of the State
Hybrid Search Score: 0.027757078409194946
----------
Id: 022c202a-f9c5-45f8-be1d-3ecaf4b6a228
Evolution
Hybrid Search Score: 0.027623284608125687
----------
Id: 73f33485-11aa-4e79-85b6-92c628620eb1
Spy Kids 3-D: Game Over
Hybrid Search Score: 0.023032408207654953
----------
Id: f5a9c3d4-39e5-4e8c-8047-66ac94b6a985
My Best Friend's Girl
Hybrid Search Score: 0.01666666753590107
----------
Id: 6222befb-fd50-402c-8fe9-d7b577afc880
The Bucket List
Hybrid Search Score: 0.016393441706895828
----------
Id: 02094432-7aca-4b5b-97c3-180d982e9062
僕のヒーローアカデミア THE MOVIE ～2人の英雄～
Hybrid Search Score: 0.016129031777381897
----------
Id: 591e75dc-b477-48dc-95a9-60be485066fa
I Don't Know How She Does It
Hybrid Search Score: 0.01587301678955555
----------
Id: efdf94dd-0b38-4ca

###  Hybrid Search
####  Let's try our more specific query again to see the difference in the score returned.

In [22]:
query = "Who are the actors in the movie Hidden Figures?"

search_client = SearchClient(
    acs_endpoint_name,
    acs_index_name,
    AzureKeyCredential(acs_api_key)
)

# You can see here that we are getting the embedding representation of the query.
vector = Vector(
    value=embeddings.embed_query(query),
    k=5,
    fields="content_vector"
)

# -----
# Notice we also fill in the search_text parameter with the query along with the vector.
# -----
results = list(search_client.search(
    search_text=query,
    include_total_count=True,
    top=10,
    vectors=[vector],
    select=["id", "content", "original_title"],
))

# Print count of total results.
print(f"Returned {len(results)} results using hybrid search.")
print("----------")
# Iterate over results and print out the id and search score.
for result in results:  
    print(f"Id: {result['id']}")
    print(f"Title: {result['original_title']}")
    print(f"Hybrid Search Score: {result['@search.score']}")
    print("----------")

Returned 10 results using hybrid search.
----------
Id: 9f50f113-ecbb-4e6c-a84e-0f464b64cc6c
Title: Hidden Figures
Hybrid Search Score: 0.03333333507180214
----------
Id: 233354af-807b-4685-812c-902a35ea4a2b
Title: Lovelace
Hybrid Search Score: 0.027108971029520035
----------
Id: 591e75dc-b477-48dc-95a9-60be485066fa
Title: I Don't Know How She Does It
Hybrid Search Score: 0.02629443258047104
----------
Id: c369ee73-1251-4e4e-bb96-21cde2305289
Title: Step Sisters
Hybrid Search Score: 0.025057602673768997
----------
Id: 0d34d588-8b70-4588-a1ea-d39caf1e1127
Title: Women
Hybrid Search Score: 0.0229242704808712
----------
Id: 6222befb-fd50-402c-8fe9-d7b577afc880
Title: The Bucket List
Hybrid Search Score: 0.016393441706895828
----------
Id: 730f545c-d963-416d-980e-01f58b46f344
Title: Norm of the North
Hybrid Search Score: 0.016129031777381897
----------
Id: 5b57538f-c14d-4081-9550-d6da1af6d0e7
Title: Enemy of the State
Hybrid Search Score: 0.01587301678955555
----------
Id: 02094432-7aca-4b

### Bringing it All Together with Retrieval Augmented Generation (RAG) + Langchain (LC)
Now that we have our Vector Store setup and data loaded, we are now ready to implement the RAG pattern using AI Orchestration. At a high-level, the following steps are required:

* Ask the question
* Create Prompt Template with inputs
* Get Embedding representation of inputted question
* Use embedded version of the question to search Azure Cognitive Search (ie. The Vector Store)
* Inject the results of the search into the Prompt Template & Execute the Prompt to get the completion

### Question to be asked

In [23]:
question = "List the movies about ships on the water."

In [24]:
# Create a prompt template with variables, note the curly braces
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    input_variables=["original_question","search_results"],
    template="""
    Question: {original_question}

    Do not use any other data.
    Only use the movie data below when responding.
    {search_results}
    """,
)


### Get Embedding for the original question

In [26]:
question_embedded=embeddings.embed_query(question)
print(question_embedded)

[0.0005168093138262972, -0.0376063676924632, -1.0562264405258914e-05, -0.020479234868726725, -0.012491808842157639, -0.0019461165370657852, -0.03666358980721698, -0.0017906236114140072, -0.003931515563627048, -0.019379326094175982, 0.013814316760701509, 0.014063105628008877, 0.009591456939764624, 0.007162494445656029, 0.012675126660808557, 0.008471908433865393, 0.019536456983480506, 0.0050707056635199985, 0.015909380073159108, -0.018475831396917202, -0.020505421798073117, -0.0218410241126128, 0.013473869915391297, -0.013853599948688948, -0.0015508371413537607, -0.005060884866523139, 0.019968562738116178, -0.02257429724986171, 0.0008437536110037692, -0.02588711424421929, 0.03420189178878016, -0.02512765417762399, 0.006939893904678979, -0.0066354552513733294, -0.00533913519349847, -0.004268689275599613, 0.000246538093372597, -0.009722398105754907, -0.0028397914850656486, -0.0017251529120035389, -0.0031311362080367927, 0.012459073783490723, -0.022888555303180295, -0.027261997884101177, -0

### Search Vector Store

In [27]:
vector = Vector(
    value=question_embedded,
    k=5,
    fields="content_vector"
)

In [28]:
results = list(search_client.search(
    search_text="",
    include_total_count=True,
    vectors=[vector],
    select=["original_title"],
))

### Build the Prompt and Execute against the Azure OpenAI to get the completion

In [29]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
response = chain.run({"original_question": question, "search_results": results})
print(response)



> Entering new LLMChain chain...
Prompt after formatting:

    Question: List the movies about ships on the water.

    Do not use any other data.
    Only use the movie data below when responding.
    [{'original_title': 'Phantom', '@search.score': 0.8349474, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}, {'original_title': 'Pirates of the Caribbean: Tales of the Code – Wedlocked', '@search.score': 0.83367515, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}, {'original_title': 'Броненосец Потёмкин', '@search.score': 0.8313832, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}, {'original_title': 'Under Siege 2: Dark Territory', '@search.score': 0.8253132, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}, {'original_title': 'America: The Motion Picture', '@search.score': 0.82525986, '@search.reranker_score': None, '@search.highlights':

### Using the `ConversationalRetrievalChain` chain   

##### Connect to the the Azure Cognitive Service

In [30]:
from langchain.vectorstores.azuresearch import AzureSearch
acs = AzureSearch(azure_search_endpoint=acs_endpoint_name,
                 azure_search_key=acs_api_key,
                 index_name=acs_index_name,
                 embedding_function=embeddings.embed_query)

In [31]:
print(acs_index_name)

movies-index-hzm


### initiate your retriever 

In [32]:
retriever = acs.as_retriever()

We create our  question-answering chat chain. In this case, we specify the condense question prompt, which converts the user’s question to a standalone question (using the chat history), in case the user asked a follow-up questio

In [33]:
# Adapt if needed
from langchain.chains import ConversationalRetrievalChain
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=acs.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)



Let’s ask a question:

In [37]:
chat_history = []
query = "Tell me the story of The Painted Veil?"
result = qa({"question": query, "chat_history": chat_history})

print("Question:", query)
print("Answer:", result["answer"])

Question: Tell me the story of The Painted Veil?
Answer: The Painted Veil is a romantic drama film released in 2006. The story revolves around a British medical doctor who finds himself in a challenging situation. He is sent to a small Chinese village to fight a cholera outbreak. However, he is also trapped in a loveless marriage to an unfaithful wife.

As the doctor works tirelessly to combat the disease and save lives, he begins to develop a deeper understanding of the local culture and the people he encounters. Along the way, he forms a connection with a young Chinese woman who helps him navigate the challenges he faces.

Amidst the backdrop of the cholera epidemic, the doctor's marriage undergoes a transformation. Both he and his wife confront their own flaws and shortcomings, leading to a journey of self-discovery and redemption. The film explores themes of love, forgiveness, and the complexities of human relationships.

"The Painted Veil" is a poignant tale that delves into the e

In [ ]:
print(result)

From where, we can also ask follow up questions:

In [38]:
chat_history = [(query, result["answer"])]
query = "What is its genre?"
result = qa({"question": query, "chat_history": chat_history})

print("Question:", query)
print("Answer:", result["answer"])

Question: What is its genre?
Answer: "The Painted Veil" belongs to the genres of Romance and Drama.


In [40]:
chat_history = [(query, result["answer"])]
query = "ok thanks, can you recomend similar movies, Please on use the provided movies data?"
result = qa({"question": query, "chat_history": chat_history})

print("Question:", query)
print("Answer:", result["answer"])

Question: ok thanks, can you recomend similar movies, Please on use the provided movies data?
Answer: Based on the genres and themes of the provided movies, here are some similar movies:

1. "Team America: World Police" (2004) - This satirical comedy film also combines action and comedy with a political theme.

2. "A Beautiful Mind" (2001) - This drama film explores mental health issues and features a protagonist dealing with schizophrenia.

3. "Out of Africa" (1985) - This romantic drama film follows a British protagonist in a foreign country, dealing with a complicated relationship and personal challenges.

4. "Source Code" (2011) - This science fiction thriller involves time travel and a protagonist trying to prevent a disaster while uncovering the truth.

Please note that these suggestions are based on the genres and themes of the provided movies and may not be exact matches in terms of plot or setting.
